In [1]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist
import pandas as pd
import nltk
import re
import math
import numpy as np
from keras.layers import Dense, Input, Flatten, Dropout, Lambda
from keras.models import Model, Sequential
from keras.utils.np_utils import to_categorical
from io import StringIO
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 


Using TensorFlow backend.


In [3]:
tokenize = lambda doc: doc.lower().split(" ")
records =[]
allrecords = [] 
allkeys =[] 
test= []
summaryall=[]
clean_data=[]
clean_data1=[]
main_words=[]
all_words1=[]
all_words2=[]
testrecords=[]
testsummary=[]
all_words3=[]
XY= pd.read_csv('RAW_DATA.csv')
data = XY["DATA"]
labels = XY["CATEGORIES"]
labels_index = {}


In [4]:
for idx, i in enumerate(XY.CATEGORIES.unique()):
    labels_index[i] = idx
cachedStopWords = set(stopwords.words("english"))
cachedStopWords.update(('english',':','?','\b0','\f0','-','arial','cell','caller','default','paragraph','-','follow -up','@',';','by:','xxxx-xxxx','details',',','patient','call','additional','{','}'))


In [5]:
for x in data[:13000]:
    message = re.sub(r"([\\])(\w+)\b",r" ",x)
    message= re.sub(r"\w*\d\w*",r" ", message)
    message = re.sub(' +',' ',message)
    message=re.sub(r"^(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4}|am|pm)$",r" ",message)
    regex = re.compile(r'''
    [\S]+:                # a key (any word followed by a colon)
    (?:
    \s                    # then a space in between
        (?!\S+:)\S+       # then a value (any word not followed by a colon)
    )+                    # match multiple values if present
    ''', re.VERBOSE)
    
    matches = regex.findall(message)
    allrecords.append(matches)


In [7]:
listclean=list(map(''.join, allrecords))
sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
sklearn_representation = sklearn_tfidf.fit_transform(listclean)


In [8]:
print(sklearn_representation.shape)
category=XY.CATEGORIES[:13000]
print(len(category))
from keras.utils.np_utils import to_categorical


(13000, 32290)
13000


In [9]:
labels = [labels_index[label] for label in category]
labels = to_categorical(np.asarray(labels))
model = Sequential()
model.add(Dense(500, input_dim=sklearn_representation.shape[1]))
model.add(Dense(len(labels_index), activation='softmax'))


In [10]:
from keras.optimizers import SGD
sgd = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(sklearn_representation.todense(), labels, validation_split=0.1, epochs=2)


Train on 11700 samples, validate on 1300 samples
Epoch 1/2
11700/11700 [==============================] - 140s 12ms/step - loss: 1.4368 - acc: 0.5929 - val_loss: 0.9923 - val_acc: 0.6769
Epoch 2/2
11700/11700 [==============================] - 119s 10ms/step - loss: 0.5435 - acc: 0.8462 - val_loss: 0.9646 - val_acc: 0.6862


In [12]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(hidden_layer_sizes=(500,), max_iter=5)



In [ ]:
clf.fit(sklearn_representation.todense(), labels)
from sklearn.metrics import accuracy_score


In [ ]:

accuracy_score(clf.predict(sklearn_representation.todense()), labels)